In [1]:
from tensorflow.python.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM,GRU
 
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
 
from sklearn.preprocessing import MinMaxScaler
 
from datetime import datetime

from binance.client import Client

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

In [2]:
client = Client('', '')

In [3]:
cmc = CoinMarketCapAPI('  ')

In [4]:
info = client.get_all_tickers()

In [5]:
'Good to check indivudual by get_symbol_info if status is trading'

'Good to check indivudual by get_symbol_info if status is trading'

In [6]:
'Filter the Info to get the list of Symbols ending with USDT'

'Filter the Info to get the list of Symbols ending with USDT'

In [7]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith('USDT')]

In [8]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [9]:
target_rank_info = [d['symbol']+'USDT' for d in rank_info if d['rank'] >= 300 and d['rank'] <= 500]

In [10]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [11]:
target_ticker_list

['FETUSDT',
 'DUSKUSDT',
 'COCOSUSDT',
 'TOMOUSDT',
 'MFTUSDT',
 'WANUSDT',
 'BANDUSDT',
 'NKNUSDT',
 'ARPAUSDT',
 'TROYUSDT',
 'BTSUSDT',
 'LTOUSDT',
 'MBLUSDT',
 'VTHOUSDT',
 'BLZUSDT',
 'KMDUSDT',
 'ANTUSDT',
 'WNXMUSDT',
 'YFIIUSDT',
 'DIAUSDT',
 'BELUSDT',
 'SUNUSDT',
 'FLMUSDT',
 'ORNUSDT',
 'UTKUSDT',
 'XVSUSDT',
 'ALPHAUSDT',
 'CTKUSDT',
 'DNTUSDT',
 'AVAUSDT',
 'REEFUSDT',
 'RIFUSDT',
 'TRUUSDT',
 'LITUSDT',
 'SFPUSDT',
 'BADGERUSDT',
 'PONDUSDT',
 'LINAUSDT',
 'PERPUSDT',
 'SUPERUSDT',
 'TKOUSDT',
 'FORTHUSDT',
 'BAKEUSDT',
 'BURGERUSDT',
 'POLSUSDT',
 'MASKUSDT',
 'XVGUSDT',
 'ATAUSDT',
 'GTCUSDT',
 'PHAUSDT',
 'MLNUSDT',
 'CLVUSDT',
 'TVKUSDT',
 'FARMUSDT',
 'ALPACAUSDT',
 'MBOXUSDT',
 'GHSTUSDT',
 'TRIBEUSDT',
 'IDEXUSDT',
 'RADUSDT',
 'BETAUSDT',
 'DARUSDT',
 'RGTUSDT',
 'MOVRUSDT',
 'VGXUSDT',
 'JASMYUSDT',
 'PYRUSDT',
 'ALCXUSDT',
 'MCUSDT',
 'ACHUSDT',
 'ANCUSDT',
 'BIFIUSDT',
 'REIUSDT']

In [ ]:
def is_candidate(filtered_klines_info):
    context_klines_info = filtered_klines_info[0:3]
    is_candidate = True
    for context_kline in context_klines_info:
        
        volume_gate = float(context_kline[5]) * float(context_kline[4]) < 200000
        percentage_gate = (float(context_kline[1]) - float(context_kline[4]))/float(context_kline[1]) * 100 < 1

        'Get the Volume and check less than 500K Volume * Close Price' 
        'The percentage inrease in context candle should not be more than 3 percentage'
        if is_candidate and volume_gate and percentage_gate:
            is_candidate = True
        else:
            is_candidate = False
    
    trigger_kline = filtered_klines_info[-1:]
    volume_gate = float(trigger_kline[0][5]) * float(trigger_kline[0][4]) > 1200000
    percentage_gate = (7 <= (float(trigger_kline[0][4]) - 
                       float(trigger_kline[0][1]))/float(trigger_kline[0][1]) * 100 <= 14)
    if is_candidate and volume_gate and percentage_gate:
        is_candidate = True
    else:
        is_candidate = False

    return is_candidate

In [ ]:
hit_ticker_dictionary = {}

In [ ]:
for target_ticker in target_ticker_list:

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_1HOUR, "30 day ago UTC") 
    length = int(len(klines))
    for x in range(0, length, 4):
        filtered_klines_info = klines[x:x+4]
        if(len(filtered_klines_info) == 4 and is_candidate(filtered_klines_info)):
            print('********** SUCCESS ************************')
            print(target_ticker)
            hit_ticker_dictionary[target_ticker] = filtered_klines_info[-1:]
            

In [ ]:
full_test_data_set = []

for key in hit_ticker_dictionary.keys():
    key_test_data_set = []
    key_test_data_set.append(key) 
    trigger_time_in_ms = hit_ticker_dictionary[key][0][0]
    key_test_data_set.append(trigger_time_in_ms) 
    open_price = hit_ticker_dictionary[key][0][4]
    key_test_data_set.append(open_price)
    analysis_kline_list = client.get_historical_klines(symbol=key, 
                                                       interval=Client.KLINE_INTERVAL_1HOUR, 
                                                       start_str=trigger_time_in_ms)[1:6]
    for sell_kline in analysis_kline_list:
        close_price = sell_kline[4]

        key_test_data_set.append(close_price)
        diff = (float(close_price) - float(open_price))
        profit = diff/float(open_price) * 100
        key_test_data_set.append(profit)
        
    full_test_data_set.append(key_test_data_set)
    

In [ ]:
full_test_data_set